## Importing Libraries

In [1]:
import time

In [2]:
import torch

print(torch.cuda.is_available())

if torch.cuda.is_available():
    print("Cuda is Available")
    device = "cuda"
else:
    print("Cuda not found")
    device = "cpu"

True
Cuda is Available


In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

/home/dbailley/.conda/envs/llama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import json
import os
import re
import requests as req
import urllib.request
# from dotenv import load_dotenv
from IPython.display import display, Markdown

In [5]:
import chromadb

from llama_index.core import Document, Settings, StorageContext, VectorStoreIndex
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.node_parser import SentenceSplitter
# from llama_index.core.response.notebook_utils import display_response
from llama_index.core.schema import MetadataMode
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.embeddings.openai import OpenAIEmbedding
# from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.chroma import ChromaVectorStore

# import openai

from utils_18 import extract_from_json, flatten_dict, replace_double_newline, format_flattened_dict, pfizer_ncts

### Environment

In [6]:
# %reload_ext watermark
# %watermark -v -p llama_index.core
# # Python implementation: CPython
# # Python version       : 3.11.7
# # IPython version      : 8.20.0
# # llama_index.core: 0.10.12

In [7]:
# Versions:  chromadb and llama-index 
!pip list | grep chromadb
!pip list | grep llama
# chromadb                                 0.4.23
# llama_cpp_python                         0.2.53
# llama-index                              0.10.20
# llama-index-agent-openai                 0.1.5
# llama-index-cli                          0.1.8
# llama-index-core                         0.10.21.post1
# llama-index-embeddings-huggingface       0.1.4
# llama-index-embeddings-nomic             0.1.6
# llama-index-embeddings-openai            0.1.6
# llama-index-indices-managed-llama-cloud  0.1.3
# llama-index-legacy                       0.9.48
# llama-index-llms-huggingface             0.1.3
# llama-index-llms-llama-cpp               0.1.3
# llama-index-llms-ollama                  0.1.2
# llama-index-llms-openai                  0.1.12
# llama-index-multi-modal-llms-openai      0.1.4
# llama-index-program-openai               0.1.4
# llama-index-question-gen-openai          0.1.3
# llama-index-readers-file                 0.1.9
# llama-index-readers-llama-parse          0.1.3
# llama-index-vector-stores-chroma         0.1.6
# llama-index-vector-stores-elasticsearch  0.1.5
# llama-parse                              0.3.8
# llamaindex-py-client                     0.1.13

chromadb                                 0.4.24
llama_cpp_python                         0.2.56
llama-index                              0.10.20
llama-index-agent-openai                 0.1.5
llama-index-cli                          0.1.9
llama-index-core                         0.10.21.post1
llama-index-embeddings-huggingface       0.1.4
llama-index-embeddings-openai            0.1.6
llama-index-indices-managed-llama-cloud  0.1.4
llama-index-legacy                       0.9.48
llama-index-llms-llama-cpp               0.1.3
llama-index-llms-openai                  0.1.9
llama-index-multi-modal-llms-openai      0.1.4
llama-index-program-openai               0.1.4
llama-index-question-gen-openai          0.1.3
llama-index-readers-file                 0.1.11
llama-index-readers-llama-parse          0.1.3
llama-index-vector-stores-chroma         0.1.6
llama-parse                              0.3.9
llamaindex-py-client                     0.1.13


### Verify API tokens are available

In [8]:
# load_dotenv() 
# # nomic_api_key = os.getenv("NOMIC_API_KEY")
# # openai_api_key = os.getenv("OPENAI_API_KEY")

# OPENAI_API_KEY = openai_api_key

### (Optional) Remove previous JSON files and Chroma DB before starting
using macOS/Linux %%bash

In [9]:
%%bash
find ./ -type f -name "*.json" -delete

In [10]:
%%bash
rm -rf chroma_db

<span style="color: blue; font-size: 30px;">Data Collection

##### Fetch data corresponding to Pfizer PLS<br>source:  https://www.pfizer.com/science/clinical-trials/plain-language-study-results-summaries/

In [11]:
def get_trial(nct_id):
    """
    Return: the JSON data for a clinical trial given its NCT ID.
    """
    trial = req.get(f"https://clinicaltrials.gov/api/v2/studies/{nct_id}")
    trial_json = trial.json()
    return trial_json

In [12]:
def get_downloaded_json(list_of_nct_id):
    """
    Downloads and saves JSON file(s) locally for reference.
    """
    downloaded_json = []
    for nct_id in list_of_nct_id:
        trial = get_trial(nct_id)
        downloaded_json.append(trial)
        with open(f"{nct_id}.json", "w") as f:
            json.dump(trial, f, indent=4)
    return downloaded_json

##### Trials to consider (Pfizer, phase 3, with p-value)

In [13]:
# list of NCT IDs for the trials
list_of_nct_id = pfizer_ncts

In [14]:
# list_of_nct_id.sort()

In [15]:
# list_of_nct_id[175:185]

In [16]:
# list_of_nct_id = list_of_nct_id[175:185]

In [17]:
# fetch the JSON data for the trials
downloaded_json = get_downloaded_json(list_of_nct_id)
# downloaded_json[0] # check
len(downloaded_json) # check

247

<span style="color: blue; font-size: 30px;">Data Exploration and Cleaning

## Data Cleaning
For each trial, extract a  subset of the data, save to a list<br>the **extract_from_json()** function is found in imported utils **(this function is WIP and can be improved)**

In [18]:
def list_from_extracted_json(downloaded_json): 
    """
    Processes and saves extracted JSON file(s) locally for review
    Return: a list of documents.
    """
    documents_list  = []
    for json_file in downloaded_json:
        extracted_json = extract_from_json(json_file)
        nct_id = json_file['protocolSection']['identificationModule']['nctId']
        save_path = f"{nct_id}_extracted.json"
        with open(save_path, "w") as f:
            json.dump(extracted_json, f, indent=4)
        documents_list.append(extracted_json)
    return documents_list

In [19]:
# create a list of documents from the extracted JSON data 
documents_list = list_from_extracted_json(downloaded_json)
len(documents_list) # check

247

### Metadata
Note: JSON files clinicaltrials.gov are heterogenous and each may contain different number of fields.

In [20]:
def max_keys(documents_list):
    """
    Identifies the document with the maximum number of keys in a list of dictionaries.
    Return: a list of keys from the document with the maximum number of keys. 
    """
    max_index, _ = max(enumerate(documents_list), key=lambda x: len(x[1].keys()))
    all_keys = list(documents_list[max_index].keys())
    return all_keys

In [21]:
# get the keys from the document that has the most keys
# use "all_keys" during creation Llama Documents
all_keys = max_keys(documents_list)
len(all_keys) # check
# all_keys # check

68

In [22]:
# all_keys

In [23]:
def adjust_metadata_keys(all_keys, keys_to_include):
    """
    To adjust the metadata keys used.
    Return: keys to exclude from list of all_keys not in list of keys_to_include.
    """
    keys_to_exclude = [key for key in all_keys if key not in keys_to_include]
    return keys_to_exclude

##### Select metadata for LLM

In [24]:
# WIP — need to explore the how these affect RAG Retrieval
llm_keys_to_incude = [
    "National Clinical Identification NCT ID",
    "Brief title",
    "Condition",
    "Conditions keywords",
    "Lead sponsor",
    "Arms group 0 intervention names",
    "Statistical Method"
]

# to exclude the keys not used by LLM
llm_keys_to_exclude = adjust_metadata_keys(all_keys, llm_keys_to_incude)
len(llm_keys_to_exclude) # check

61

##### Select metadata for Embedding

In [25]:
# WIP — need to explore the how these affect RAG Retrieval 
# for simplicity, this is the same as llm_keys_to_exclude (in this example)
embedding_keys_to_incude = [
    "National Clinical Identification NCT ID",
    "Brief title",
    "Condition",
    "Conditions keywords",
    "Lead sponsor",
    "Arms group 0 intervention names",
    "Statistical Method",
]

# to exclude the keys not used by embedding
embedding_keys_to_exclude = adjust_metadata_keys(all_keys, llm_keys_to_incude)
len(embedding_keys_to_exclude) # check

61

<span style="color: blue; font-size: 30px;">LlamaIndex

##### Select LLM  type

In [26]:
model_url = 'https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf'

In [27]:
llm = LlamaCPP(
    model_url=model_url,
    temperature=0,
    max_new_tokens=512,
    context_window=8190, 
    model_kwargs={"n_gpu_layers": -1},
    verbose=True
)


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla V100-PCIE-16GB, compute capability 7.0, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /tmp/llama_index/models/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: -

##### Select Embedding type

In [28]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

##### Global settings (Embedding & LLM)

In [29]:
Settings.llm = llm
Settings.embed_model = embed_model

### **Creating & Loading LlamaIndex Documents**

##### Example of format to load into LlamaIndex

In [30]:
# example = documents_list[0]
# # apply functions from utils
# processed_example = format_flattened_dict(flatten_dict(example))
# display(Markdown(processed_example)) # check

In [31]:
# example # check

In [32]:
# NOTE:  metata data must be one of (str, int, float, None)
def create_llama_docs(documents_list):
    """
    Converts a list of trial documents into LlamaIndex Document objects.
    """
    
    llama_documents = []  
    for trial in documents_list:
        # apply functions from utils to flatten JSON and create content similar to the example above
        content_text = format_flattened_dict(flatten_dict(trial))

        llama_document = Document(
            text=content_text, 
            metadata=trial, 
            excluded_llm_metadata_keys=llm_keys_to_exclude, #<== adjust?, TBD
            excluded_embed_metadata_keys=embedding_keys_to_exclude , #<== adjust?, TBD
            metadata_template="{key}=>{value}",
            text_template="Metadata:\n{metadata_str}\n===========================\nContent: \n{content}"
        )
        llama_documents.append(llama_document)  
    return llama_documents

In [33]:
# Create LlamaIndex Document objects
llama_documents = create_llama_docs(documents_list)
len(llama_documents) # check

247

In [34]:
# Example — LLM sees this:
print(llama_documents[0].get_content(metadata_mode=MetadataMode.LLM))

Metadata:
National Clinical Identification NCT ID=>NCT00036270
Brief title=>Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen for 2.5 to 3 Years Followed By Exemestane
Lead sponsor=>Pfizer
Condition=>['Breast Neoplasms']
Conditions keywords=>oof, this data not available
Arms group 0 intervention names=>['Drug: exemestane (Aromasin)']
Statistical Method=>Log Rank
Content: 
"National Clinical Identification NCT ID": "NCT00036270",
"Organization study identification": "971-ONC-0028-081",
"EudraCT number": "A5991026",
"Organization": "Pfizer",
"Organization class": "INDUSTRY",
"Brief title": "Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen for 2.5 to 3 Years Followed By Exemestane",
"Official title": "Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen For 2.5- 3 Years Followed By Exemestane (Aromasin) For A Total Of 5 Years As Adjuvant Therapy For Postmenopausal, Receptor Positive, Node Negative o

In [35]:
# Example — Embedding sees this:
print(llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED))

Metadata:
National Clinical Identification NCT ID=>NCT00036270
Brief title=>Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen for 2.5 to 3 Years Followed By Exemestane
Lead sponsor=>Pfizer
Condition=>['Breast Neoplasms']
Conditions keywords=>oof, this data not available
Arms group 0 intervention names=>['Drug: exemestane (Aromasin)']
Statistical Method=>Log Rank
Content: 
"National Clinical Identification NCT ID": "NCT00036270",
"Organization study identification": "971-ONC-0028-081",
"EudraCT number": "A5991026",
"Organization": "Pfizer",
"Organization class": "INDUSTRY",
"Brief title": "Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen for 2.5 to 3 Years Followed By Exemestane",
"Official title": "Randomized Phase III Study Of Exemestane (Aromasin) For 5 Years Versus Tamoxifen For 2.5- 3 Years Followed By Exemestane (Aromasin) For A Total Of 5 Years As Adjuvant Therapy For Postmenopausal, Receptor Positive, Node Negative o

### **Chunking** — creating LlamaIndex *Nodes*

In [36]:
def create_nodes(llama_documents):
    """
    Generates and embeds nodes from Llama documents.
    """
    parser = SentenceSplitter(chunk_size=8190,chunk_overlap=0) # <== adjust from default
    nodes = parser.get_nodes_from_documents(llama_documents)
    for node in nodes:
        node_embedding = embed_model.get_text_embedding(
            node.get_content(metadata_mode=MetadataMode.EMBED)
        )
        node.embedding = node_embedding
    return nodes

In [37]:
# create nodes from Llama documents
nodes = create_nodes(llama_documents)

### **Storing** Nodes — example with **Chroma Vector Database**

In [38]:
# Chroma DB collection name
COLLECTION_NAME = "RAG_OPENSOURCE"

db = chromadb.PersistentClient(path="chroma_db")
print(f"Looking for the {COLLECTION_NAME} collection in the database..." )
if COLLECTION_NAME not in [col.name for col in db.list_collections()]:
    print(f"{COLLECTION_NAME} collection WAS NOT FOUND in Chroma DB, creating...")
    chroma_collection = db.create_collection(COLLECTION_NAME)
    print("Creating vector store...")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    print("Creating vector store index")
    VectorStoreIndex(
        nodes=nodes,
        storage_context=storage_context,
        store_nodes_override=True
    )
    record_count = chroma_collection.count()
    print(f"record count: {record_count}")   
    
else:
    print(f"{COLLECTION_NAME} collection WAS FOUND in Chroma DB")
    COLLECTION_NAME = db.get_collection(COLLECTION_NAME)
    vector_store = ChromaVectorStore(chroma_collection=COLLECTION_NAME)
    print("Restoring vector store index from the collection...")
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model,
        store_nodes_override=True
    )
    record_count = COLLECTION_NAME.count()
    print(f"record count: {record_count}")

Looking for the RAG_OPENSOURCE collection in the database...
RAG_OPENSOURCE collection WAS NOT FOUND in Chroma DB, creating...
Creating vector store...
Creating vector store index
record count: 248


###  **Creating Index** from stored Nodes

In [39]:
index = VectorStoreIndex.from_vector_store(vector_store)  

### (optional) add record(s)
no error thrown if records are dupes

In [40]:
print(f"record_count as is: {record_count}")

record_count as is: 248


In [41]:
# new unrelateds trial
list_of_nct_id = [
    "NCT00108953", # Carcinoma, Hepatocellular
    "NCT02883049", # Acute Lymphoblastic Leukemia and Ph-Like TKI Sensitive Mutations (large JSON)
]

downloaded_json = get_downloaded_json(list_of_nct_id)
documents_list = list_from_extracted_json(downloaded_json)

In [42]:
# using previous functions 
llama_documents = create_llama_docs(documents_list)
nodes = create_nodes(llama_documents)
index.insert_nodes(nodes)

In [43]:
try:
    print(f"new DB record count: {chroma_collection.count()}")
    record_count = chroma_collection.count()
except:
    print(f"established DB record count: {COLLECTION_NAME.count()}")
    record_count = COLLECTION_NAME.count()

new DB record count: 251


### **Querying** — example with ChatBot

In [44]:
start_time = time.time()

In [45]:
# chat_engine.reset() 

In [46]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=10000)

chat_engine = index.as_chat_engine(
#     chat_mode="condense_plus_context",
    chat_mode="condense_question",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Emulate a chatbot and answer concisely."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        """\nInstruction: Provide a response based only on chat history and the context above. 
        When there is no context or chat history, just say so. 
        Never respond with information outside the scope of the context or chat history.
        """
    ),
    verbose=False,
)

In [47]:
query = "Provide brief title(s) for studies involving newborn babies and Sildenafil?"
response = chat_engine.chat(query)
print(response.response)


llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      12.71 ms /    22 runs   (    0.58 ms per token,  1730.92 tokens per second)
llama_print_timings: prompt eval time =     877.53 ms /    81 tokens (   10.83 ms per token,    92.30 tokens per second)
llama_print_timings:        eval time =     230.42 ms /    21 runs   (   10.97 ms per token,    91.14 tokens per second)
llama_print_timings:       total time =    1184.16 ms /   102 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      95.64 ms /   155 runs   (    0.62 ms per token,  1620.66 tokens per second)
llama_print_timings: prompt eval time =    3111.13 ms /  6328 tokens (    0.49 ms per token,  2033.99 tokens per second)
llama_print_timings:        eval time =    2395.57 ms /   154 runs   (   15.56 ms per token,    64.29 tokens per second)
llama_print_timings:       total time =    6262.39 ms /  6482 


Based on the context information provided, two potential study titles investigating the effects of Sildenafil on newborn babies could be:
1. "A Study to Evaluate the Safety and Efficacy of IV Sildenafil in the Treatment of Newborns with Persistent Pulmonary Hypertension of the Newborn"
2. "A Randomized, Double-Blind, Placebo-Controlled Study of Oral Sildenafil in the Treatment of Pulmonary Arterial Hypertension in Newborns"
These titles are based on the conditions and interventions mentioned in the context information, but it's important to note that the actual study titles may differ from these suggestions.


In [48]:
query = "Provide the NCT ID for the first study you mention."
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      31.20 ms /    52 runs   (    0.60 ms per token,  1666.72 tokens per second)
llama_print_timings: prompt eval time =     112.87 ms /   254 tokens (    0.44 ms per token,  2250.40 tokens per second)
llama_print_timings:        eval time =     547.08 ms /    51 runs   (   10.73 ms per token,    93.22 tokens per second)
llama_print_timings:       total time =     838.85 ms /   305 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =       6.80 ms /    11 runs   (    0.62 ms per token,  1618.36 tokens per second)
llama_print_timings: prompt eval time =    3122.27 ms /  6358 tokens (    0.49 ms per token,  2036.34 tokens per second)
llama_print_timings:        eval time =     155.29 ms /    10 runs   (   15.53 ms per token,    64.40 tokens per second)
llama_print_timings:       to

 NCT01720524


In [49]:
query = "What was the enrollment count for this study?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      33.46 ms /    55 runs   (    0.61 ms per token,  1643.56 tokens per second)
llama_print_timings: prompt eval time =     134.35 ms /   282 tokens (    0.48 ms per token,  2099.00 tokens per second)
llama_print_timings:        eval time =     581.23 ms /    54 runs   (   10.76 ms per token,    92.91 tokens per second)
llama_print_timings:       total time =     906.40 ms /   336 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      33.95 ms /    55 runs   (    0.62 ms per token,  1619.98 tokens per second)
llama_print_timings: prompt eval time =    3120.93 ms /  6361 tokens (    0.49 ms per token,  2038.17 tokens per second)
llama_print_timings:        eval time =     837.78 ms /    54 runs   (   15.51 ms per token,    64.46 tokens per second)
llama_print_timings:       to

59 participants were enrolled in the clinical trial with NCT ID NCT01720524 investigating the safety and efficacy of IV Sildenafil in treating Persistent Pulmonary Hypertension of the Newborn.


In [50]:
query = "What was EudraCT number of this trial?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      25.80 ms /    43 runs   (    0.60 ms per token,  1666.34 tokens per second)
llama_print_timings: prompt eval time =     142.14 ms /   355 tokens (    0.40 ms per token,  2497.52 tokens per second)
llama_print_timings:        eval time =     453.93 ms /    42 runs   (   10.81 ms per token,    92.53 tokens per second)
llama_print_timings:       total time =     745.17 ms /   397 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =       9.31 ms /    15 runs   (    0.62 ms per token,  1610.82 tokens per second)
llama_print_timings: prompt eval time =    3121.21 ms /  6349 tokens (    0.49 ms per token,  2034.15 tokens per second)
llama_print_timings:        eval time =     217.28 ms /    14 runs   (   15.52 ms per token,    64.43 tokens per second)
llama_print_timings:       to

2012-002619-24


In [51]:
query = "Briefly, what conditions are mentioned in this trial involving Sildenafil?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      36.60 ms /    61 runs   (    0.60 ms per token,  1666.85 tokens per second)
llama_print_timings: prompt eval time =     157.71 ms /   395 tokens (    0.40 ms per token,  2504.61 tokens per second)
llama_print_timings:        eval time =     650.64 ms /    60 runs   (   10.84 ms per token,    92.22 tokens per second)
llama_print_timings:       total time =    1024.96 ms /   455 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      56.17 ms /    91 runs   (    0.62 ms per token,  1620.11 tokens per second)
llama_print_timings: prompt eval time =    3124.09 ms /  6367 tokens (    0.49 ms per token,  2038.03 tokens per second)
llama_print_timings:        eval time =    1397.31 ms /    90 runs   (   15.53 ms per token,    64.41 tokens per second)
llama_print_timings:       to


The clinical trial with NCT ID NCT01720524 and EudraCT number 2012-002619-24 is studying the use of Sildenafil in newborns with persistent pulmonary hypertension of the newborn (PPHN) or hypoxic respiratory failure (HRF) and at risk of PPHN.


In [52]:
query = "What statistical method used this trial?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      44.99 ms /    75 runs   (    0.60 ms per token,  1667.19 tokens per second)
llama_print_timings: prompt eval time =     171.01 ms /   500 tokens (    0.34 ms per token,  2923.74 tokens per second)
llama_print_timings:        eval time =     812.04 ms /    74 runs   (   10.97 ms per token,    91.13 tokens per second)
llama_print_timings:       total time =    1248.18 ms /   574 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      50.59 ms /    82 runs   (    0.62 ms per token,  1620.84 tokens per second)
llama_print_timings: prompt eval time =    3134.12 ms /  6381 tokens (    0.49 ms per token,  2035.98 tokens per second)
llama_print_timings:        eval time =    1258.30 ms /    81 runs   (   15.53 ms per token,    64.37 tokens per second)
llama_print_timings:       to

 Analysis of covariance (ANCOVA) was employed in the clinical trial investigating the safety and efficacy of IV Sildenafil in treating Persistent Pulmonary Hypertension of the Newborn with NCT ID NCT01720524 and EudraCT number 2012-002619-24.


In [53]:
query = "What p-value was reported?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      46.12 ms /    77 runs   (    0.60 ms per token,  1669.59 tokens per second)
llama_print_timings: prompt eval time =     271.76 ms /   596 tokens (    0.46 ms per token,  2193.08 tokens per second)
llama_print_timings:        eval time =     838.45 ms /    76 runs   (   11.03 ms per token,    90.64 tokens per second)
llama_print_timings:       total time =    1388.71 ms /   672 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =       4.32 ms /     7 runs   (    0.62 ms per token,  1621.50 tokens per second)
llama_print_timings: prompt eval time =    3129.66 ms /  6383 tokens (    0.49 ms per token,  2039.52 tokens per second)
llama_print_timings:        eval time =      93.25 ms /     6 runs   (   15.54 ms per token,    64.35 tokens per second)
llama_print_timings:       to

0.9850


In [54]:
query = "What does this p-value suggest about the study in question? Provide an overview of how researchers use p-value in such studies."
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      37.73 ms /    63 runs   (    0.60 ms per token,  1669.85 tokens per second)
llama_print_timings: prompt eval time =     274.06 ms /   638 tokens (    0.43 ms per token,  2327.94 tokens per second)
llama_print_timings:        eval time =     686.29 ms /    62 runs   (   11.07 ms per token,    90.34 tokens per second)
llama_print_timings:       total time =    1185.60 ms /   700 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =     139.40 ms /   226 runs   (    0.62 ms per token,  1621.28 tokens per second)
llama_print_timings: prompt eval time =    3125.66 ms /  6369 tokens (    0.49 ms per token,  2037.65 tokens per second)
llama_print_timings:        eval time =    3503.18 ms /   225 runs   (   15.57 ms per token,    64.23 tokens per second)
llama_print_timings:       to


The reported p-value of 0.9850 indicates that there was no statistically significant difference between the placebo and sildenafil groups in the study investigating the safety and efficacy of IV Sildenafil in treating Persistent Pulmonary Hypertension of the Newborn. A p-value is a probability value that helps researchers determine the significance of the results of a statistical test. In clinical trials, a p-value less than 0.05 (commonly referred to as the 'significance level') is considered statistically significant, meaning that the observed difference between the treatment and control groups is unlikely to have occurred by chance alone. However, a p-value greater than 0.05 does not necessarily mean that there is no difference between the groups, but rather that the observed difference may have occurred by chance. Researchers interpret p-values as a measure of the strength of evidence against the null hypothesis, which in this case was that there was no difference between the plac

In [55]:
query = "Based on this p-value, would researchers recommend the treatment?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      25.19 ms /    42 runs   (    0.60 ms per token,  1667.06 tokens per second)
llama_print_timings: prompt eval time =     331.88 ms /   884 tokens (    0.38 ms per token,  2663.58 tokens per second)
llama_print_timings:        eval time =     462.44 ms /    41 runs   (   11.28 ms per token,    88.66 tokens per second)
llama_print_timings:       total time =     947.80 ms /   925 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =     105.54 ms /   171 runs   (    0.62 ms per token,  1620.21 tokens per second)
llama_print_timings: prompt eval time =    3118.25 ms /  6348 tokens (    0.49 ms per token,  2035.76 tokens per second)
llama_print_timings:        eval time =    2645.79 ms /   170 runs   (   15.56 ms per token,    64.25 tokens per second)
llama_print_timings:       to


The p-value of 0.9850 indicates that there was no statistically significant difference between the placebo and sildenafil groups in the treatment of Persistent Pulmonary Hypertension of the Newborn based on the reported study. Therefore, based on this study alone, it cannot be recommended to use sildenafil over placebo for treating this condition. However, it's important to note that this study had a small sample size and further research with larger sample sizes and longer follow-up periods may be needed to confirm these findings. Additionally, other studies and clinical guidelines may support the use of sildenafil in treating Persistent Pulmonary Hypertension of the Newborn, and healthcare professionals should consult these resources for the most up-to-date and comprehensive information.


In [56]:
query = "Remind me, what are we discussing?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =      14.37 ms /    24 runs   (    0.60 ms per token,  1669.80 tokens per second)
llama_print_timings: prompt eval time =     460.57 ms /  1071 tokens (    0.43 ms per token,  2325.37 tokens per second)
llama_print_timings:        eval time =     262.55 ms /    23 runs   (   11.42 ms per token,    87.60 tokens per second)
llama_print_timings:       total time =     815.10 ms /  1094 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =     128.34 ms /   208 runs   (    0.62 ms per token,  1620.64 tokens per second)
llama_print_timings: prompt eval time =    3110.16 ms /  6330 tokens (    0.49 ms per token,  2035.26 tokens per second)
llama_print_timings:        eval time =    3218.44 ms /   207 runs   (   15.55 ms per token,    64.32 tokens per second)
llama_print_timings:       to


Our conversation revolves around two clinical studies, one investigating the safety and efficacy of intravenous (IV) Sildenafil in treating persistent pulmonary hypertension of the newborn (PPNH) or hypoxic respiratory failure at risk of PPNH, and the other examining the effects of oral Sildenafil on pulmonary arterial hypertension in children aged 1-17 years. Both studies were sponsored by Pfizer and employed randomized, double-blind, placebo-controlled designs. The first study compared the time on inhaled nitric oxide treatment between the placebo and IV Sildenafil groups, while the second study aimed to assess the efficacy, safety, and pharmacokinetics of 16 weeks of chronic treatment with oral Sildenafil in three different doses compared to placebo, using peak volume of oxygen consumed as the primary outcome measure.


In [57]:
end_time = time.time()
print(f"runtime:  {end_time - start_time:.0f} seconds")

runtime:  64 seconds


In [58]:
chat_engine.reset() 

In [59]:
query = "Remind me, what are we discussing?"
response = chat_engine.chat(query)
print(response.response)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =       5.73 ms /    10 runs   (    0.57 ms per token,  1746.11 tokens per second)
llama_print_timings: prompt eval time =      84.88 ms /    69 tokens (    1.23 ms per token,   812.90 tokens per second)
llama_print_timings:        eval time =      95.37 ms /     9 runs   (   10.60 ms per token,    94.37 tokens per second)
llama_print_timings:       total time =     215.04 ms /    78 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     878.15 ms
llama_print_timings:      sample time =     100.11 ms /   162 runs   (    0.62 ms per token,  1618.17 tokens per second)
llama_print_timings: prompt eval time =    2253.39 ms /  4945 tokens (    0.46 ms per token,  2194.47 tokens per second)
llama_print_timings:        eval time =    2330.43 ms /   161 runs   (   14.47 ms per token,    69.09 tokens per second)
llama_print_timings:       to

 The topic of our current discussion is a clinical trial with the title "Cross Over Convenience And Preference Study Of New Mark VII Compared To Genotropin Pen In Pediatric And Adult Subjects" and the identifier NCT01112865, which was conducted by Pfizer and focused on assessing the convenience and preference of the new Mark VII pen compared to the current Genotropin pen in pediatric and adult subjects with growth hormone deficiency. The study was a multicenter, open-label, randomized two arm cross over study, where participants used the Mark VII pen for 2 months followed by the current pen for 2 months, and vice versa, and reported their perception of ease of use through a questionnaire.


### Using **CallbackManager & LlamaDebugHandler**

In [60]:
STOPPING HERE 

SyntaxError: invalid syntax (141466560.py, line 1)

In [ ]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

# initiaalize debug handler and save to Settings
llm_debug = LlamaDebugHandler()
callback_manager = CallbackManager([llm_debug])
Settings.callback_manager = callback_manager

In [ ]:
# helper function
def print_in_out(in_out):
	print(in_out[0][0].payload['messages'][0])
	print("\n\n")
	print(in_out[0][0].payload['messages'][1])

# whenever query_engine.query() is called
query_engine = index.as_query_engine()
query = "How many NCT ID related to Leukemia? Which are they?"
response = query_engine.query(query)
print(response)

# print LLM debug info 
in_out = llm_debug.get_llm_inputs_outputs()
print_in_out(in_out)